In [29]:

            
#def xgbclassifiercv(max_depth, learning_rate, n_estimators, gamma, min_child_weight, max_delta_step, subsample,
#              colsample_bytree, silent = True, nthread = -1, seed = 1234):
    
#    return cross_val_score(XGBClassifier(max_depth = int(max_depth), learning_rate = learning_rate,
#                                         n_estimators = int(n_estimators), silent = silent,
#                                         nthread = nthread, gamma = gamma, min_child_weight = min_child_weight,
#                                         max_delta_step = max_delta_step, subsample = subsample,
#                                         colsample_bytree = colsample_bytree, seed = seed,
#                                         objective = 'binary:logistic'),
#                           X_train,
#                           y_train,
#                           'log_loss',
#                           n_jobs=-1,
#                           cv=10).mean()


#xgbclassifierBO = BayesianOptimization(xgbclassifiercv,
#                                 {'max_depth': (5, 12),
#                                  'learning_rate': (0.001, 0.5),
#                                  'n_estimators': (50, 1000),
#                                  'gamma': (1., 0.01),
#                                  'min_child_weight': (1, 100),
#                                  'max_delta_step': (0, 10),
#                                  'subsample': (0.01, 0.9),
#                                  'colsample_bytree' :(0.5, 0.99)
#                                 })


#print('Maximizing...')

#xgbclassifierBO.maximize()
#print('-'*53)
#print('Final Results')
#print('XGBClassifier: %f' % xgbclassifierBO.res['max']['max_val'])







#def extratreesclassifiercv(n_estimators, max_depth, max_features):
    
#    return cross_val_score(ExtraTreesClassifier(max_depth = int(max_depth), 
#                                                n_estimators = int(n_estimators), 
#                                                max_features = int(max_features)),
#                           X_train,
#                           y_train,
#                           'log_loss',
#                           n_jobs=-1,
#                           cv=10).mean()


#extratreesclassifierBO = BayesianOptimization(extratreesclassifiercv,
#                                 {'max_depth': (5, 12),
#                                  'n_estimators': (50, 1000),
#                                  'max_features' :(5, 130)
#                                 })


#print('Maximizing...')

#extratreesclassifierBO.maximize()
#print('-'*53)
#print('Final Results')
#print('ExtraTrees: %f' % extratreesclassifierBO.res['max']['max_val'])


import pandas as pd
import numpy as np
from bayes_opt import BayesianOptimization
from sklearn.cross_validation import cross_val_score, train_test_split
from xgboost import XGBClassifier
from sklearn.ensemble import ExtraTreesClassifier
import scipy as sp
from sknn.mlp import Classifier, Layer
from sklearn.grid_search import RandomizedSearchCV

def get_data():
    print('Load data...')

    DATA_DIR = "/Users/patrickkennedy/Desktop/Data_Science_MISC/Kaggle"
    train = pd.read_csv(DATA_DIR + "/BNP_Paribas/train.csv")
    test = pd.read_csv(DATA_DIR + "/BNP_Paribas/test.csv")

    target = train['target'].values

    train = train.drop(['ID','target'],axis=1)
    id_test = test['ID'].values
    test = test.drop(['ID'],axis=1)

    print('Clearing...')
    for (train_name, train_series), (test_name, test_series) in zip(train.iteritems(),test.iteritems()):
        if train_series.dtype == 'O':
            #for objects: factorize
            train[train_name], tmp_indexer = pd.factorize(train[train_name])
            test[test_name] = tmp_indexer.get_indexer(test[test_name])
            #but now we have -1 values (NaN)
        else:
            #for int or float: fill NaN
            tmp_len = len(train[train_series.isnull()])
            if tmp_len>0:
                #print "mean", train_series.mean()
                train.loc[train_series.isnull(), train_name] = -9999 #train_series.mean()
            #and Test
            tmp_len = len(test[test_series.isnull()])
            if tmp_len>0:
                test.loc[test_series.isnull(), test_name] = -9999 #train_series.mean()  #TODO

            
    #Spliting data into train and test sets.
    X, X_test, y, y_test = train_test_split(train, target, test_size=0.2)
    
    #Spliting train data into training and validation sets.
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25)
    
    return X, X_test, y, y_test, X_train, X_valid, y_train, y_valid
    
    
def random_search_nn():
    nn = Classifier(layers = [Layer("Rectifier",units=10),
                              Layer("Softmax")])

    params={'learning_rate': sp.stats.uniform(0.001, 0.05),
            'hidden0__units': sp.stats.randint(4, 20),
            'hidden0__type': ["Rectifier", "Sigmoid", "Tanh"]}

    return RandomizedSearchCV(nn,param_distributions=params,n_iter=5,cv=10,scoring='log_loss',n_jobs=-1)

    



if __name__ == '__main__':
    
    X, X_test, y, y_test, X_train, X_valid, y_train, y_valid = get_data()
    rsnn = random_search_nn()
    rsnn.fit(X_train, y_train)
    report(random_search.grid_scores_)

Load data...
Clearing...


ImportError: [joblib] Attempting to do parallel computing without protecting your import on a system that does not support forking. To use parallel-computing in a script, you must protect your main loop using "if __name__ == '__main__'". Please see the joblib documentation on Parallel for more information

In [ ]:
#CV=10

#####################################
#XGBClassifier.....
#Optimization finished with maximum: -0.467777, at position: {'colsample_bytree': 0.77638333498678636, 'learning_rate': 0.030567867858705199, 'max_delta_step': 4.6626180513766657, 'min_child_weight': 57.354121041109124, 'n_estimators': 477.72777553323016, 'subsample': 0.8069399976204783, 'max_depth': 6.6059347713535548, 'gamma': 0.2966938071810209}.
#Time taken: 499 minutes and 40.785203 seconds.
#-----------------------------------------------------
#Final Results
#XGBClassifier: -0.467777
#####################################
#ExtraTreesClassifier....
#Optimization finished with maximum: -0.472478, at position: {'n_estimators': 108.03076614800023, 'max_features': 130.0, 'max_depth': 12.0}.
#Time taken: 350 minutes and 16.242177 seconds.
#-----------------------------------------------------
#Final Results
#ExtraTrees: -0.472478
#####################################
#NN....
#####################################
#RandomForest...
#####################################









In [ ]:
#add NN, RF to the bayes_opt to get at their params
#run the script with calibrations
#when i am ready add the whole model (using the same weighting as with the training set)